# VacationPy
----


In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import pprint

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
part1_csv_file = "../WeatherPy/random_cities_weather_data_file.csv"
random_cities_weather_df = pd.read_csv(part1_csv_file)
random_cities_weather_df = random_cities_weather_df.drop(columns=["Unnamed: 0"])

In [21]:

random_cities_weather_df.head(2)

,City_Name,Country_Code,Latitude,Longititude,Temprature,Humidity,cloudiness,wind_speed
0,saint-philippe,re,-56.221530,66.735628,76.82,65,20,23.04
1,dukuhturi,id,-6.836002,108.847204,87.12,64,69,6.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
cities_humidity_df = random_cities_weather_df["Humidity"].astype(float)
cities_humidity_max_df = cities_humidity_df.max()
cities_location_df = random_cities_weather_df[["Latitude","Longititude"]]

In [22]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities_location_df, weights=cities_humidity_df,
                                 dissipating=False, max_intensity=cities_humidity_max_df , point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### New DataFrame fitting weather criteria
- Tempratuture greater than 70 and less than 80
- wind speed less than 10mph
- zero cloudiness

In [6]:
condition1 = random_cities_weather_df["cloudiness"] == 0
condition2 = random_cities_weather_df["Temprature"] >70 
condition3 = random_cities_weather_df["Temprature"] <=80
condition4 = random_cities_weather_df["wind_speed"] <=10
mask_df = random_cities_weather_df[condition1 & condition2 & condition3 & condition4 ].reset_index(drop=True)
mask_df

,City_Name,Country_Code,Latitude,Longititude,Temprature,Humidity,cloudiness,wind_speed
0,tessalit,ml,22.315007,3.824591,78.24,17,0,6.04
1,qena,eg,26.801000,32.653905,75.20,38,0,2.24
2,topolobampo,mx,25.098967,-109.859953,79.00,92,0,2.95
3,komatipoort,za,-24.763552,31.773187,73.00,71,0,5.44
4,fuling,cn,29.863516,107.761067,71.04,47,0,4.16
5,buraydah,sa,27.664997,43.319203,75.20,33,0,3.36
6,rundu,na,-17.481829,20.596792,76.84,18,0,6.13
7,sohag,eg,23.528975,28.323022,75.20,36,0,6.93
8,safaga,eg,26.484647,35.019427,75.20,57,0,8.05
9,baghdad,iq,32.129849,44.936125,78.80,44,0,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#hotel_df = random_cities_weather_df[condition1 & condition2 & condition3 & condition4 ]
hotel_df = mask_df.copy()
hotel_df["Hotel_Name"] =''
hotel_df.reset_index(inplace= True,drop = True)
hotellist= []

In [8]:
hotellist= []
i = 0
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]["Latitude"]
    lng = hotel_df.loc[i]["Longititude"]
    params = {
            "location": f"{lat},{lng}",
            "language": "en",
            "radius": 5000,
            "types": "hotel",         
            "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params = params)
    data = response.json()
    if data['status'] == "OK":
        #data['results'][0]['name']
        #print("OK if")
        xx = data['results'][0]['name']
        if xx != '':
            #print(f'print {xx}')
            hotellist.append(xx)
        
        #print(f'For {i} row value is : {xxxx}')
    else:
        hotellist.append('None')
        #print(f'Appended : Blank')
        

#hotellist[2] = "Non English response"
hotel_df["Hotel_Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')


In [13]:
hotel_df1 = hotel_df.copy()
condition1 = hotel_df1["Hotel_Name"] != "None"
hotel_df1 = hotel_df1[condition1]
hotel_df1.head(15)

,City_Name,Country_Code,Latitude,Longititude,Temprature,Humidity,cloudiness,wind_speed,Hotel_Name
0,tessalit,ml,22.315007,3.824591,78.24,17,0,6.04,Abalessa
4,fuling,cn,29.863516,107.761067,71.04,47,0,4.16,Chongqing
5,buraydah,sa,27.664997,43.319203,75.20,33,0,3.36,قطبة ابوصالح
6,rundu,na,-17.481829,20.596792,76.84,18,0,6.13,Luenge National Park
9,baghdad,iq,32.129849,44.936125,78.80,44,0,9.17,Dagharah
10,huangchuan,cn,32.566828,115.192746,71.98,30,0,5.82,Xinyang
12,ankang,cn,32.646908,109.081060,74.07,39,0,1.16,Ankang


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City_Name}</dd>
<dt>Country</dt><dd>{Country_Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df1.iterrows()]
locations = hotel_df1[["Latitude", "Longititude"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))